In electricity markets, renewable energy producers (e.g., wind farms) must submit bids ahead of time (e.g., 12-36 hours in advance). However, due to uncertainty in wind power generation, their actual output may deviate from the bid, leading to imbalance costs.

Thus, the bid:

- Should be close to expected generation to maximize profits.
- Must consider imbalance penalties from over- or under-production.
- Can be optimized using statistical models (ARMA, GPR, MLE).

In [ ]:
!pip3.12 install polars

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement fast_excel (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip3.12 install --upgrade pip
ERROR: No matching distribution found for fast_excel


In [12]:
!pip3.12 install fastexcel

Defaulting to user installation because normal site-packages is not writeable
  Using cached fastexcel-0.12.1-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
  Using cached pyarrow-19.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached fastexcel-0.12.1-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
Using cached pyarrow-19.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.1 MB)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip3.12 install --upgrade pip


In [7]:
import sys
print(sys.version)
print(sys.executable)


3.12.4 (main, Oct 27 2024, 18:41:26) [GCC 11.4.0]
/usr/local/bin/python3.12


## Data Loading

Wind Farms Generation Data: Three wind farms located
in Australia including Taralga Wind Farm (TARALGA1) in New
SouthWales(NSW1)zone,BaldHillsWindFarm(BALDHWF1)in
Victoria (VIC1) zone and Woolnorth Studland Bay / Bluff Point
WindFarm (WOOLNTH1) inTasmania (TAS1) zone arechosen for
the purpose of our case studies. The installed capacities of these
wind farms are 107 MW, 107MW and 140MW, respectively. The
proposed method is applied to forecast the hourly generation of
these wind farms for a period of nine months spanning from the
first day of April to the last day of December of the year 2018.
All wind farms generation data with a 5-minute time resolution
are obtained from [46].

Electricity Market Assumptions: The participants in the
 day-ahead electricity market are supposed to submit their bids
 before 12 : 00 a.m. on the current day for each hour of the next
 operating day, therefore, forecasts are from 12 to 36 hours look
ahead times. The electricity prices for the day-ahead market and
 for the deviations are taken from [47] and [48], respectively. The
 day-ahead prices, as denoted by rd in equations (12) and (13)
 of Section II-D, are assumed as the values reported in [47] for
 each bidding zone including NSW1, VIC1 and TAS1. To estimate
 the imbalance unit costs in these equations, i.e. λ+ and λ−,as  certain proportion of the day-ahead price, we normalised the
 four-day average values before the test day reported in [48] and
 considered them as hourly values of α+ and α− and computed
 λ+
 d =(1−α+)rd and λ−
 d =(1+α−)rd.


In [1]:
import polars as pl
import numpy as np

#### Wind Farms data one year-Australia

filenameA = "forecast_code/WP_5min_output_2018.xlsx"

# Read data using polars (assuming Excel format is supported)
subsetA1 = pl.read_excel(filenameA, sheet_name="TARALGA1")[1:]  # Skip header row
subsetA3 = pl.read_excel(filenameA, sheet_name="BALDHWF1")[1:]  # Skip header row 
subsetA4 = pl.read_excel(filenameA, sheet_name="WOOLNTH1")[1:]  # Skip header row

# Function to convert 5-min data to hourly by averaging every 12 rows
def aggregate_hourly(df):
    arr = df.to_numpy().flatten()  # Convert to NumPy array and flatten
    num_rows = len(arr) // 12 * 12  # Ensure it's a multiple of 12
    arr = arr[:num_rows]  # Truncate extra rows
    hourly_avg = arr.reshape(-1, 12).mean(axis=1)  # Compute hourly averages
    return pl.DataFrame({"hourly_output": hourly_avg})  # Ensure only 1 column

# Apply function to each dataset
WP1_hourly = aggregate_hourly(subsetA1)
WP3_hourly = aggregate_hourly(subsetA3)
WP4_hourly = aggregate_hourly(subsetA4)

# Exclude negative values (similar to MATLAB filtering)
WP1_hourly = WP1_hourly.with_columns(
    pl.when(pl.col("hourly_output") < 0).then(0).otherwise(pl.col("hourly_output")).alias("hourly_output")
)
WP3_hourly = WP3_hourly.with_columns(
    pl.when(pl.col("hourly_output") < 0).then(0).otherwise(pl.col("hourly_output")).alias("hourly_output")
)
WP4_hourly = WP4_hourly.with_columns(
    pl.when(pl.col("hourly_output") < 0).then(0).otherwise(pl.col("hourly_output")).alias("hourly_output")
)

# Store wind power data in a dictionary similar to MATLAB struct
WP = {
    "name": ["Tarlaga", "Bald Hills", "Woolnorth"],
    "capacity": [107, 107, 140],
    "location": ["NSW", "Victoria", "Tasmania"],
    "date": ["2018"],
    "hourly_output": [WP1_hourly.to_numpy(), WP3_hourly.to_numpy(), WP4_hourly.to_numpy()],
}

print(WP['hourly_output'][0].shape)

# Load Excel file
filenameB = "forecast_code/da_half_hour_price_2018.xlsx"

# Read data using polars
subsetB1 = pl.read_excel(filenameB, sheet_name="NSW")[1:]  # Skip header row
subsetB2 = pl.read_excel(filenameB, sheet_name="VIC")[1:]  # Skip header row
subsetB3 = pl.read_excel(filenameB, sheet_name="TAS")[1:]  # Skip header row


# to long format
subsetB1 = subsetB1.unpivot(value_name="NSW")[:,1:]
subsetB2 = subsetB2.unpivot(value_name="VIC")[:,1:]
subsetB3 = subsetB3.unpivot(value_name="TAS")[:,1:]

# drop rows with value 0
subsetB1 = subsetB1.filter(subsetB1['NSW'] != 0)
subsetB2 = subsetB2.filter(subsetB2['VIC'] != 0)
subsetB3 = subsetB3.filter(subsetB3['TAS'] != 0)

# replace negative value with 0.01
subsetB1 = subsetB1.with_columns(
    pl.when(pl.col("NSW") < 0).then(0.01).otherwise(pl.col("NSW")).alias("NSW")
)
subsetB2 = subsetB2.with_columns(
    pl.when(pl.col("VIC") < 0).then(0.01).otherwise(pl.col("VIC")).alias("VIC")
)
subsetB3 = subsetB3.with_columns(
    pl.when(pl.col("TAS") < 0).then(0.01).otherwise(pl.col("TAS")).alias("TAS")
)


# Function to aggregate half-hourly prices to hourly averages
def aggregate_half_hourly(df, column_name):
    df = df.with_columns(pl.Series("index", range(len(df))))  # Create an index column
    df = df.with_columns((pl.col("index") // 2).alias("hourly_index"))  # Group every 2 rows
    df = df.group_by("hourly_index", maintain_order = True).agg(pl.col(column_name).mean().alias(column_name))
    return df.drop("hourly_index")  # Drop index column after aggregation

# Apply function to each dataset
da_NSW = aggregate_half_hourly(subsetB1, "NSW").to_numpy()
da_VIC = aggregate_half_hourly(subsetB2, "VIC").to_numpy()
da_TAS = aggregate_half_hourly(subsetB3, "TAS").to_numpy()

# Store market data in a dictionary similar to MATLAB struct
DA_price = {
    "Day_ahead_Price": {
        "NSW": da_NSW,
        "VIC": da_VIC,
        "TAS": da_TAS
    }
}

# Print the final cleaned and aggregated prices
print(DA_price["Day_ahead_Price"]["NSW"].shape)

# Load all 12 sheets from the Excel file
filenameC = "forecast_code/Imb_15min_price_2018.xls"

# Read first sheet and select columns **by index**
df = pl.read_excel(filenameC, sheet_id=1)
# Select columns by position and cast to float
MIP = df[2:, 6:7].rename({"__UNNAMED__6": "MIP"}).with_columns(pl.col("MIP").cast(pl.Float64))
MDP = df[2:, 7:8].rename({"__UNNAMED__7": "MDP"}).with_columns(pl.col("MDP").cast(pl.Float64))

# Append data from the other 11 sheets and ensure float conversion
for i in range(2, 13):
    df = pl.read_excel(filenameC, sheet_id=i)
    
    MIP = MIP.vstack(df[2:, 6:7].rename({"__UNNAMED__6": "MIP"}).with_columns(pl.col("MIP").cast(pl.Float64)))
    MDP = MDP.vstack(df[2:, 7:8].rename({"__UNNAMED__7": "MDP"}).with_columns(pl.col("MDP").cast(pl.Float64)))


#**Step 1: Replace negative values in MDP with zero**
MDP = MDP.with_columns(pl.when(pl.col("MDP") < 0).then(0).otherwise(pl.col("MDP")))

# **Step 2: Convert 15-minute data to hourly by averaging every 4 rows**
def aggregate_15min_to_hourly(df, column_name):
    df = df.with_columns(pl.Series("index", range(len(df))))  # Create an index column
    df = df.with_columns((pl.col("index") // 4).alias("hourly_index"))  # Group every 4 rows
    df = df.group_by("hourly_index", maintain_order=True).agg(pl.col(column_name).mean().alias(column_name))
    return df.drop("hourly_index")  # Drop index column after aggregation

# Apply function to both datasets
hourly_MIP = aggregate_15min_to_hourly(MIP, "MIP")
hourly_MDP = aggregate_15min_to_hourly(MDP, "MDP")

# **Step 3: Store in a dictionary similar to MATLAB struct**
IMB_price = {
    "neg": hourly_MDP.to_numpy(),
    "pos": hourly_MIP.to_numpy()
}

# Print the final cleaned and aggregated prices
print(IMB_price["neg"].shape)
print(IMB_price["pos"].shape)

(8759, 1)
(8754, 1)
(8757, 1)
(8757, 1)


## Forecasting Wind Power Generation

In [ ]:
import numpy as np

def compute_daily_imbalance(hourly_MID, hourly_MDP):
    """
    Computes the daily average imbalance prices for each hour of the day.

    Args:
        hourly_MID (np.array): Historical positive imbalance prices.
        hourly_MDP (np.array): Historical negative imbalance prices.

    Returns:
        tuple: (daily_MID, daily_MDP)
    """
    daily_MID = np.zeros(24)  # Initialize array for daily positive imbalance prices
    daily_MDP = np.zeros(24)  # Initialize array for daily negative imbalance prices

    for h in range(24):
        daily_MID[h] = np.mean(hourly_MID[h:len(hourly_MID)-24+h:24])  # Mimic MATLAB indexing
        daily_MDP[h] = np.mean(hourly_MDP[h:len(hourly_MDP)-24+h:24])

    return daily_MID, daily_MDP

def process_imbalance_prices(which_WP, which_day, DA_price, IMB_price, useful_start, hist_step):
    """
    Processes imbalance prices, calculates adjusted imbalance prices, 
    and prepares the imbalance price ratios.

    Args:
        which_WP (int): Wind power plant index.
        which_day (int): Forecasting day.
        DA_price (dict): Day-ahead market prices.
        IMB_price (dict): Imbalance price data.
        useful_start (int): Start hour for useful predictions.
        hist_step (int): Number of historical hours used for training.

    Returns:
        tuple: (imbalance_cell, ratio)
    """

    # Compute first_h_day
    first_h_day = (which_day * 24) - (useful_start - 1) - hist_step

    # Step 1: Historical Imbalance Calculation (last 4 days before `which_day`)
    hist_imb = which_day * 24 - 4 * 24  # 4 days lookback

    # Day-ahead prices for the forecast day (24 hours)
    da_hourly_after = DA_price["Day_ahead_Price"]["NSW"][(which_day * 24):((which_day * 24) + 24)]

    # Historical Imbalance Prices
    hourly_MDP = IMB_price["neg"][hist_imb:(which_day * 24)]
    hourly_MID = IMB_price["pos"][hist_imb:(which_day * 24)]

    # Compute daily average imbalance prices
    daily_MID, daily_MDP = compute_daily_imbalance(hourly_MID, hourly_MDP)

    # Step 2: Adjusted Imbalance Prices
    lamda_pos_after = daily_MID
    lamda_neg_after = daily_MDP

    typ = lamda_pos_after / np.max(lamda_pos_after)  # Normalization
    tyn = lamda_neg_after / np.max(lamda_neg_after)

    lamda_pos_after = (1 - typ) * da_hourly_after
    lamda_neg_after = (1 + tyn) * da_hourly_after

    lamda_pos = da_hourly_after - lamda_pos_after
    lamda_neg = lamda_neg_after - da_hourly_after

    # Ratio of Imbalance Prices
    ratio = lamda_pos / (lamda_pos + lamda_neg)

    # Step 3: Target Forecasting Day Imbalance Prices
    lamda_pos_after2 = IMB_price["neg"][(which_day * 24):((which_day * 24) + 24)]
    lamda_neg_after2 = IMB_price["pos"][(which_day * 24):((which_day * 24) + 24)]

    typ2 = lamda_pos_after2 / np.max(lamda_pos_after2)
    tyn2 = lamda_neg_after2 / np.max(lamda_neg_after2)

    lamda_pos_after22 = (1 - typ2) * da_hourly_after
    lamda_neg_after22 = (1 + tyn2) * da_hourly_after

    lamda_pos_after2 = da_hourly_after - lamda_pos_after22
    lamda_neg_after2 = lamda_neg_after22 - da_hourly_after

    # Step 4: Store in an Imbalance Cell Structure
    imbalance_cell = {
        "lamda_pos": lamda_pos,
        "lamda_neg": lamda_neg,
        "lamda_pos_after2": lamda_pos_after2,
        "lamda_neg_after2": lamda_neg_after2,
    }

    return imbalance_cell, ratio



def train_test_set(which_WP, first_h_day, WP, forecast_step, hist_step):
    """
    Prepares training and test sets for wind power forecasting.

    Args:
        which_WP (int): Index of wind power plant.
        first_h_day (int): Start hour for training.
        WP (dict): Wind power dataset.
        forecast_step (int): Forecast horizon (hours).
        hist_step (int): Number of historical hours used for training.

    Returns:
        tuple: (x_train, y_train, x_test, y_test)
    """

    # Define training and testing time windows
    a1 = first_h_day  # Start of training period
    a2 = first_h_day + hist_step  # End of training period (3 months)
    b1 = a2  # Start of testing period
    b2 = b1 + forecast_step  # End of testing period (forecast horizon)

    # Load wind power data
    WP_cap = WP["capacity"][which_WP]  # Wind power capacity
    WP_hourly = WP["hourly_output"][which_WP]  # Hourly wind power output

    # Training set (input: time, output: normalized wind power)
    x_train = np.arange(1, (a2 - a1) + 1)  # Time index for training
    y_train = (WP_hourly[a1:a2] / WP_cap).flatten()  # Ensure 1D

    # Test set (input: time, output: normalized wind power)
    x_test = np.arange((a2 - a1) + 1, (a2 - a1) + forecast_step + 1)  # Time index for testing
    y_test = (WP_hourly[b1:b2] / WP_cap).flatten()  # Ensure 1D

    return x_train, y_train, x_test, y_test


# defined in yearanalysisWP1.m
which_WP = 0 # 0 for Taralga, 1 for Bald Hills, 2 for Woolnorth
useful_start = 1
hist_step = 3*30*24 # historical data used for training (3 months)
forecast_step = 36
useful_start=13 
useful_end=36
days = list(range(91, 365))  # Days from 90 to 364
which_day = days[0]  # Forecasting day

# Defined in Main_Execute.m
first_h_day = which_day * 24 - (useful_start - 1) - hist_step
WP_cap = WP["capacity"][which_WP]  # Wind power capacity

# Compute imbalance prices
imbalance_cell, ratio = process_imbalance_prices(which_WP, which_day, DA_price, IMB_price, useful_start, hist_step)

# Print results
# print(imbalance_cell["lamda_pos"])
# print(imbalance_cell["lamda_neg"])
# print(imbalance_cell["lamda_pos"].shape)
# print("Ratio Shape:", ratio.shape)


# Prepare training and test sets
x_train, y_train, x_test, y_test = train_test_set(which_WP, first_h_day, WP, forecast_step, hist_step)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


(2160,) (2160,) (36,) (36,)


In [27]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

def Arma2(y_train, which_day, forecast_step):
    """
    Implements an ARMA model to predict wind power outputs.

    Args:
        y_train (np.array): Training data (normalized wind power).
        which_day (int): Forecasting day.
        forecast_step (int): Number of steps to forecast.

    Returns:
        tuple: (y_pred_arma2, y_sd_arma2, bad_days)
    """
    num_y = 72  # AR lags
    num_e = 36  # MA lags

    try:
        # Define ARMA model (ARMA(p=72, q=36))
        model = ARIMA(y_train, order=(num_y, 0, num_e))  # No differencing (d=0)
        
        # Fit the model
        fitted_model = model.fit()
        
        # Forecast the next `forecast_step` steps
        forecast_results = fitted_model.get_forecast(steps=forecast_step)
        
        y_pred_arma2 = forecast_results.predicted_mean
        y_sd_arma2 = forecast_results.se_mean  # Standard error
        
        bad_days = []
    
    except Exception as e:
        print(f"ARMA Model failed for day {which_day}: {e}")
        
        # If the model fails, return random predictions
        y_pred_arma2 = np.random.rand(forecast_step)
        y_sd_arma2 = np.random.rand(forecast_step)
        bad_days = [which_day]

    return y_pred_arma2, y_sd_arma2, bad_days


# ARMA Agent Predict WP Output
y_pred_arma2, y_sd_arma2, bad_days = Arma2(y_train, which_day, forecast_step)
print(y_pred_arma2.shape, y_sd_arma2.shape, bad_days)
print(y_pred_arma2)


/home/tlorans/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/tlorans/.local/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


KeyboardInterrupt: 

## Mapping Forecast Distributions to Probability Values

## Prediction Market



Clearance mechanisms in prdiction markets:
- Continuous Double Auction (CDA)
- Automated Market Maker (AMM)

with CD mechanism, buyers and selelrs of the contracts offer bid and ask prices and trade happens when the offered prices match. Main problem is the lack of liquidity because the bid-ask spreads often remain wide. Possible solution is the use of AMM to be always available to take the other side of an order of buying or selling, therefore providing liquidity to the market

Logarithmic Market Scoring Rule (LMSR): arbitratege free amm with continous price function and payoff values known at the time of the trade where the price of shares reflects the probability of the outcome.

The cost and price of contracts (shares) in LMSR are given by:

\begin{equation}
\pi_r (b, \pi_c, q) = \frac{1}{(1 + \frac{1/\pi_c - 1}{\exp(q / b)})}
\end{equation}

\begin{equation}
C(b, \pi_c, q) = b \ln (\pi_c (exp(q/b) - 1) +1)
\end{equation}

where $b$ is the AMM parameter which controls the available liquidity in the market, $q$ is the quantity of shares to be trader $\pi_c$ is the current market price and finally $\pi_r(b, \pi_c, q)$ and $C(b, \pi_c, q)$ are the resulting price and the cost function respectively. 

The first initial price of shares is set by the AMM as $\pi_c = \pi_0$. In previous equations, positive values of $q$ indicate buying shares while negative values of $q$ correspond to selling shares. Therefore, buying shares increases the market price while selling shares decreses the price.
